### Download dataset from kaggle and unzip it
[Large License Plate Dataset](https://www.kaggle.com/datasets/fareselmenshawii/large-license-plate-dataset/data)

In [1]:
!pip install kaggle

^C


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/82.7 kB ? eta -:--:--
     ---------------------------------------- 82.7/82.7 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
   ---------------------------------------- 162.8/162.8 kB 9.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ---------------------------------------- 78.2/78.2 kB ? eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105796 sha256=df05e0672bae52fd1600cba987f7884845f836b50c853d71c91ee4da37fb2

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [8]:
!kaggle datasets download -d fareselmenshawii/large-license-plate-dataset

Dataset URL: https://www.kaggle.com/datasets/fareselmenshawii/large-license-plate-dataset
License(s): CC0-1.0
100% 2.34G/2.35G [00:28<00:00, 207MB/s]
100% 2.35G/2.35G [00:28<00:00, 87.5MB/s]


In [9]:
import zipfile
import os

def unzip_and_create_folder(zip_file_path, extract_to_folder):
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)

zip_file_path = 'large-license-plate-dataset.zip'  # Replace with your zip file path
extract_to_folder = 'Dataset'  # Replace with your desired extract folder path

unzip_and_create_folder(zip_file_path, extract_to_folder)

#### Split data into 10 parts for faster training time

In [16]:
import shutil

base_dir = 'Dataset'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

dir_list = ['test','train','val']

num_parts = 10

for dir in dir_list:
    os.makedirs(os.path.join(images_dir, dir), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, dir), exist_ok=True)

    all_images = os.listdir(os.path.join(images_dir, dir))
    all_labels = os.listdir(os.path.join(labels_dir, dir))

    all_images.sort()
    all_labels.sort()

    part_sizes = len(all_images) // num_parts

    for i in range(num_parts):
        targetImageDir = os.makedirs(os.path.join(base_dir, 'part_'+str(i+1), 'images', dir), exist_ok=True)
        targetLabelDir = os.makedirs(os.path.join(base_dir, 'part_'+str(i+1), 'labels', dir), exist_ok=True)

        part_images = all_images[i*part_sizes:(i+1)*part_sizes]
        part_labels = all_labels[i*part_sizes:(i+1)*part_sizes]

        for img in part_images:
            shutil.copy(os.path.join(images_dir, dir, img), os.path.join(base_dir, 'part_'+str(i+1), 'images', dir))

        for lbl in part_labels:
            shutil.copy(os.path.join(labels_dir, dir, lbl), os.path.join(base_dir, 'part_'+str(i+1), 'labels', dir))


### Training

In [18]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")

results = model.train(data="Dataset/detectorData.yaml", epochs=50, imgsz=640, save=True)

Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/detectorData.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/dataset/part_1/labels/train... 2547 images, 4 backgrounds, 0 corrupt: 100%|██████████| 2547/2547 [00:03<00:00, 714.63it/s] 

train: New cache created: /content/dataset/part_1/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/dataset/part_1/labels/val... 107 images, 0 backgrounds, 0 corrupt: 100%|██████████| 107/107 [00:00<00:00, 763.31it/s]

...

      50/50      2.48G      1.091       0.54     0.9875          4        640: 100%|██████████| 160/160 [01:09<00:00,  2.32it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.36it/s]

                   all        107        150      0.889      0.827      0.879      0.557



50 epochs completed in 1.103 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.2MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.2.54 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.19it/s]


                   all        107        150      0.871      0.809      0.868      0.557
Speed: 0.4ms preprocess, 3.6ms inference, 0.0ms loss, 7.8ms postprocess per image
Results saved to runs/detect/train5
